In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#import tensorflow as tf
#tf.test.is_built_with_cuda(), tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import pandas

ModuleNotFoundError: No module named 'pandas._libs.interval'

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13171821529451520541
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9383706624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11069033451163854801
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [8]:
from pathlib import Path
import imghdr

i = 0

data_dir = "pizza types"
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
            i = i + 1
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
            i = i + 1
            
if i != 0:
        print("Number of unsupported images: ", i)


In [6]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

DATA_DIR = 'Manually Sorted Pizza'
CLASSES = ['Black Olive', 'Cheese', 'Hawaiian', 'Pepperoni', 'Taco']
CLASS_MAP = {CLASSES[i] : i for i in range(len(CLASSES)) }

image_paths = []

for dirpath, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        image_path = os.path.join(dirpath, filename)

        # Try to load the image
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image for jpg conversion: {image_path}")
            continue

        # Check if the image is already in JPG format
        if not image_path.lower().endswith('.jpg'):
            # Create a new path with the .jpg extension
            new_image_path = os.path.splitext(image_path)[0] + '.jpg'

            # Try to write the image in JPG format
            success = cv2.imwrite(new_image_path, image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

            if not success:
                print(f"Error converting to JPG: {image_path}")
                continue

            # Remove the old image
            os.remove(image_path)

            # Update the image path to the new JPG path
            image_paths.append(new_image_path)
        else:
            # The image is already in JPG format
            image_paths.append(image_path)

In [7]:
import os
import cv2

# Changes all images to .jpg format and resizes them to 256x256. Images are also in RGB format
#Rev 2: added an index to both avoid file name collisions and to find some issues, does not affect resized images - JT


def clean_data(data_dir):
    i = 0
    for dirpath, _, filenames in os.walk(data_dir):
        for filename in filenames:
            if not filename.lower().endswith('.jpg'):
                old_image_path = os.path.join(dirpath, filename)
                image = cv2.imread(old_image_path)
                resized_image = cv2.resize(image, (256,256))

                new_image_path = os.path.splitext(old_image_path)[0] + f'_{i}.jpg'
                cv2.imwrite(new_image_path, resized_image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

                os.remove(old_image_path)
            else:
                image_path = os.path.join(dirpath, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, (256,256))

                cv2.imwrite(image_path, resized_image)

            i = i + 1



                
DATA_DIR = 'Manually Sorted Pizza'
clean_data(DATA_DIR)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, input_size):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, input_size//2, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(input_size//2, input_size//4, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(input_size//4, input_size//8, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        # Calculate the size of the fully connected layer dynamically
        # based on the adjusted input channels and downsampling from convolutional layers
        self.fc_input_size = self.calculate_fc_input_size(input_size)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self.fc_input_size, 64)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(64, 5)

    def calculate_fc_input_size(self, input_size):
        # Function to calculate the size of the fully connected layer
        # based on the adjusted input channels and downsampling from convolutional layers
        dummy_input = torch.ones(1, 3, input_size, input_size)
        dummy_output = self.forward_conv(dummy_input)
        return dummy_output.view(1, -1).size(1)
    
    def forward_conv(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        return x

    def forward(self, x):
        x = self.forward_conv(x)
        x = self.flatten(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

def make_cnn(input_size, learning_rate):
    model = CNN(input_size)

    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    return model, loss_function, optimizer